## 1. Setup Environment

In [ ]:
# Install dependencies (jalankan jika di Google Colab)
!pip install -q tensorflow pandas numpy scikit-learn pyyaml tqdm matplotlib seaborn

In [ ]:
# Import libraries
import os
import re
import json
import random
import pickle
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_curve, auc, precision_recall_curve,
    accuracy_score, precision_score, recall_score, f1_score
)

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Dropout,
    Bidirectional, Conv1D, MaxPooling1D, GRU
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 2. Data Generator - Generate Log Sintetis

In [ ]:
class SystemLogGenerator:
    """
    Generator untuk membuat data log sistem sintetis.
    """
    
    def __init__(self, seed: int = 42, anomaly_ratio: float = 0.1):
        self.seed = seed
        self.anomaly_ratio = anomaly_ratio
        random.seed(seed)
        np.random.seed(seed)
        
        # Template log normal
        self.normal_templates = [
            # SSH/Authentication logs
            "Accepted password for {user} from {ip} port {port} ssh2",
            "session opened for user {user} by (uid=0)",
            "session closed for user {user}",
            "New session {session_id} of user {user}",
            
            # System logs
            "systemd[1]: Started {service}.service",
            "systemd[1]: Stopped {service}.service",
            "systemd[1]: Reloading {service}.service",
            "kernel: [{timestamp}] {device}: link up",
            
            # Application logs
            "[INFO] Application started successfully",
            "[INFO] Processing request from {ip}",
            "[INFO] Database connection established",
            "[INFO] Cache refreshed successfully",
            "[DEBUG] Request completed in {latency}ms",
            
            # Web server logs
            "{ip} - - [{datetime}] \"GET {path} HTTP/1.1\" 200 {bytes}",
            "{ip} - - [{datetime}] \"POST {path} HTTP/1.1\" 200 {bytes}",
            
            # Database logs
            "Connection received: host={ip} user={user} database={database}",
            "Query executed successfully in {latency}ms",
        ]
        
        # Template log anomali
        self.anomaly_templates = {
            'brute_force': [
                "Failed password for invalid user {user} from {suspicious_ip} port {port} ssh2",
                "Failed password for {user} from {suspicious_ip} port {port} ssh2",
                "authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost={suspicious_ip}",
                "PAM: Authentication failure for {user} from {suspicious_ip}",
                "error: maximum authentication attempts exceeded for {user} from {suspicious_ip}",
            ],
            'privilege_escalation': [
                "sudo: {user} : command not allowed ; TTY=pts/0 ; PWD=/home/{user} ; USER=root ; COMMAND=/bin/bash",
                "ALERT: Unauthorized sudo attempt by {user}",
                "su[{pid}]: FAILED su for root by {user}",
                "kernel: Possible privilege escalation attempt detected",
            ],
            'malware_indicator': [
                "ALERT: Suspicious process detected: {malware_name} (PID: {pid})",
                "WARNING: Outbound connection to known malicious IP: {suspicious_ip}",
                "ALERT: Reverse shell connection attempt to {suspicious_ip}:{port}",
                "kernel: Suspicious kernel module loaded: {module_name}",
            ],
            'suspicious_network': [
                "iptables: DROPPED: IN=eth0 SRC={suspicious_ip} DST={ip} PROTO=TCP DPT={port}",
                "DDoS attack detected from {suspicious_ip} - rate limit exceeded",
                "Port scan detected from {suspicious_ip} - multiple ports targeted",
                "[WARNING] Unusual traffic pattern from {suspicious_ip}",
            ],
            'system_error': [
                "kernel: Out of memory: Kill process {pid} ({process}) score {score}",
                "CRITICAL: Disk /dev/sda1 is full (100% used)",
                "ERROR: Service {service} crashed unexpectedly (exit code: {exit_code})",
                "kernel: CPU{cpu_id}: Temperature above threshold, cpu clock throttled",
                "[CRITICAL] Database connection pool exhausted",
            ],
        }
        
        # Data untuk template
        self.users = ['admin', 'root', 'user', 'guest', 'www-data', 'mysql', 'nginx']
        self.services = ['nginx', 'apache2', 'mysql', 'postgresql', 'redis', 'docker', 'sshd']
        self.paths = ['/api/users', '/api/data', '/login', '/dashboard', '/admin', '/static/js/app.js']
        self.databases = ['production', 'analytics', 'users', 'logs']
        self.malware_names = ['cryptominer', 'backdoor', 'trojan', 'rootkit', 'keylogger']
        self.processes = ['java', 'python', 'node', 'apache2', 'nginx', 'mysql']
        
    def _generate_ip(self, is_suspicious: bool = False) -> str:
        if is_suspicious:
            suspicious_ranges = ['185.220.101', '45.33.32', '104.248.50', '192.42.116']
            return f"{random.choice(suspicious_ranges)}.{random.randint(1, 254)}"
        return f"192.168.{random.randint(1, 10)}.{random.randint(1, 254)}"
    
    def _generate_timestamp(self, base_time: datetime = None) -> str:
        if base_time is None:
            base_time = datetime.now()
        delta = timedelta(seconds=random.randint(0, 86400))
        return (base_time - delta).strftime("%Y-%m-%d %H:%M:%S")
    
    def _fill_template(self, template: str, is_anomaly: bool = False) -> str:
        replacements = {
            '{user}': random.choice(self.users),
            '{ip}': self._generate_ip(False),
            '{suspicious_ip}': self._generate_ip(True),
            '{port}': str(random.randint(1024, 65535)),
            '{service}': random.choice(self.services),
            '{path}': random.choice(self.paths),
            '{database}': random.choice(self.databases),
            '{bytes}': str(random.randint(100, 50000)),
            '{latency}': str(random.randint(1, 500)),
            '{pid}': str(random.randint(1000, 99999)),
            '{session_id}': str(random.randint(1, 1000)),
            '{timestamp}': f"{random.randint(0, 9999)}.{random.randint(100000, 999999)}",
            '{device}': f"eth{random.randint(0, 3)}",
            '{datetime}': self._generate_timestamp(),
            '{malware_name}': random.choice(self.malware_names),
            '{module_name}': f"suspicious_mod_{random.randint(1, 100)}",
            '{process}': random.choice(self.processes),
            '{score}': str(random.randint(100, 1000)),
            '{exit_code}': str(random.choice([1, -1, 137, 139, 255])),
            '{cpu_id}': str(random.randint(0, 7)),
        }
        
        result = template
        for key, value in replacements.items():
            result = result.replace(key, value)
        return result
    
    def generate_normal_log(self) -> str:
        template = random.choice(self.normal_templates)
        return self._fill_template(template, is_anomaly=False)
    
    def generate_anomaly_log(self, anomaly_type: str = None):
        if anomaly_type is None:
            anomaly_type = random.choice(list(self.anomaly_templates.keys()))
        template = random.choice(self.anomaly_templates[anomaly_type])
        return self._fill_template(template, is_anomaly=True), anomaly_type
    
    def generate_logs(self, num_logs: int, include_labels: bool = True) -> pd.DataFrame:
        num_anomaly = int(num_logs * self.anomaly_ratio)
        num_normal = num_logs - num_anomaly
        
        logs = []
        labels = []
        anomaly_types = []
        
        # Generate normal logs
        print(f"Generating {num_normal} normal logs...")
        for _ in tqdm(range(num_normal)):
            logs.append(self.generate_normal_log())
            labels.append(0)
            anomaly_types.append('normal')
        
        # Generate anomaly logs
        print(f"Generating {num_anomaly} anomaly logs...")
        for _ in tqdm(range(num_anomaly)):
            log, atype = self.generate_anomaly_log()
            logs.append(log)
            labels.append(1)
            anomaly_types.append(atype)
        
        # Create DataFrame
        df = pd.DataFrame({
            'log_message': logs,
            'label': labels,
            'anomaly_type': anomaly_types
        })
        
        # Shuffle
        df = df.sample(frac=1, random_state=self.seed).reset_index(drop=True)
        
        return df

In [ ]:
# Generate data
generator = SystemLogGenerator(seed=42, anomaly_ratio=0.1)
df = generator.generate_logs(num_logs=10000)

print("\n" + "="*50)
print("STATISTIK DATASET")
print("="*50)
print(f"Total logs: {len(df)}")
print(f"Normal logs: {len(df[df['label'] == 0])} ({len(df[df['label'] == 0])/len(df)*100:.1f}%)")
print(f"Anomaly logs: {len(df[df['label'] == 1])} ({len(df[df['label'] == 1])/len(df)*100:.1f}%)")
print("\nDistribusi Jenis Anomali:")
print(df[df['label'] == 1]['anomaly_type'].value_counts())

In [ ]:
# Lihat sample data
print("\n📊 Sample Log Normal:")
for log in df[df['label'] == 0]['log_message'].head(5):
    print(f"  🟢 {log[:80]}...")

print("\n📊 Sample Log Anomali:")
for log in df[df['label'] == 1]['log_message'].head(5):
    print(f"  🔴 {log[:80]}...")

## 3. Preprocessing

In [ ]:
class LogPreprocessor:
    """
    Preprocessor untuk membersihkan dan normalisasi log.
    """
    
    def __init__(self):
        # Regex patterns
        self.ip_pattern = re.compile(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b')
        self.timestamp_pattern = re.compile(r'\d{4}-\d{2}-\d{2}[T ]\d{2}:\d{2}:\d{2}')
        self.pid_pattern = re.compile(r'\[\d+\]|\(PID:\s*\d+\)|pid=\d+')
        self.port_pattern = re.compile(r'port\s+\d+|:\d{2,5}')
        self.hex_pattern = re.compile(r'0x[0-9a-fA-F]+')
        self.number_pattern = re.compile(r'\b\d+\b')
    
    def clean_log(self, log: str) -> str:
        # Lowercase
        log = log.lower()
        
        # Replace patterns
        log = self.ip_pattern.sub('<IP>', log)
        log = self.timestamp_pattern.sub('<TIMESTAMP>', log)
        log = self.pid_pattern.sub('<PID>', log)
        log = self.hex_pattern.sub('<HEX>', log)
        
        # Remove special characters (keep basic punctuation)
        log = re.sub(r'[^a-zA-Z0-9\s<>_\-./]', ' ', log)
        
        # Normalize whitespace
        log = ' '.join(log.split())
        
        return log
    
    def preprocess_dataframe(self, df: pd.DataFrame, log_column: str = 'log_message') -> pd.DataFrame:
        df = df.copy()
        df['cleaned_log'] = df[log_column].apply(self.clean_log)
        return df

In [ ]:
# Preprocessing
preprocessor = LogPreprocessor()
df = preprocessor.preprocess_dataframe(df)

print("Contoh hasil preprocessing:")
for i in range(3):
    print(f"\nOriginal: {df['log_message'].iloc[i][:70]}...")
    print(f"Cleaned:  {df['cleaned_log'].iloc[i][:70]}...")

## 4. Tokenisasi

In [ ]:
class LogTokenizer:
    """
    Tokenizer untuk mengkonversi log ke sequences.
    """
    
    def __init__(self, max_words: int = 10000, max_length: int = 50, oov_token: str = '<OOV>'):
        self.max_words = max_words
        self.max_length = max_length
        self.oov_token = oov_token
        
        self.tokenizer = Tokenizer(
            num_words=max_words,
            oov_token=oov_token,
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
            lower=True
        )
        
        self.vocabulary_size = None
        self.is_fitted = False
    
    def fit(self, texts):
        self.tokenizer.fit_on_texts(texts)
        self.vocabulary_size = min(len(self.tokenizer.word_index) + 1, self.max_words)
        self.is_fitted = True
        return self
    
    def transform(self, texts):
        sequences = self.tokenizer.texts_to_sequences(texts)
        padded = pad_sequences(sequences, maxlen=self.max_length, padding='post', truncating='post')
        return padded
    
    def fit_transform(self, texts):
        self.fit(texts)
        return self.transform(texts)

In [ ]:
# Tokenisasi
MAX_WORDS = 10000
MAX_LENGTH = 50

tokenizer = LogTokenizer(max_words=MAX_WORDS, max_length=MAX_LENGTH)
X = tokenizer.fit_transform(df['cleaned_log'].tolist())
y = df['label'].values

print(f"Vocabulary size: {tokenizer.vocabulary_size}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nSample sequence: {X[0][:20]}...")

## 5. Split Data

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining - Normal: {sum(y_train==0)}, Anomaly: {sum(y_train==1)}")
print(f"Test - Normal: {sum(y_test==0)}, Anomaly: {sum(y_test==1)}")

## 6. Build Model LSTM

In [ ]:
def build_lstm_model(vocab_size, embedding_dim=128, lstm_units=64, max_length=50, dropout_rate=0.3):
    """
    Build LSTM model untuk deteksi anomali.
    """
    inputs = Input(shape=(max_length,), name='input')
    
    # Embedding layer
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='embedding')(inputs)
    
    # LSTM layers
    x = LSTM(lstm_units, return_sequences=True, name='lstm_1')(x)
    x = Dropout(dropout_rate, name='dropout_1')(x)
    x = LSTM(lstm_units // 2, name='lstm_2')(x)
    
    # Dense layers
    x = Dense(32, activation='relu', name='dense')(x)
    x = Dropout(dropout_rate, name='dropout_2')(x)
    
    # Output
    outputs = Dense(1, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name='LSTM_AnomalyDetector')
    
    return model

In [ ]:
# Build model
EMBEDDING_DIM = 128
LSTM_UNITS = 64
DROPOUT_RATE = 0.3

model = build_lstm_model(
    vocab_size=tokenizer.vocabulary_size,
    embedding_dim=EMBEDDING_DIM,
    lstm_units=LSTM_UNITS,
    max_length=MAX_LENGTH,
    dropout_rate=DROPOUT_RATE
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

model.summary()

In [ ]:
# Visualisasi arsitektur model
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, dpi=100)

## 7. Training Model

In [ ]:
# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

# Training
EPOCHS = 20
BATCH_SIZE = 32

print("\n" + "="*50)
print("TRAINING MODEL")
print("="*50)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

## 8. Visualisasi Training

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Loss
axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0, 0].set_title('Model Loss', fontsize=14)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0, 1].set_title('Model Accuracy', fontsize=14)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
axes[1, 0].plot(history.history['precision'], label='Train Precision', linewidth=2)
axes[1, 0].plot(history.history['val_precision'], label='Val Precision', linewidth=2)
axes[1, 0].set_title('Model Precision', fontsize=14)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Recall
axes[1, 1].plot(history.history['recall'], label='Train Recall', linewidth=2)
axes[1, 1].plot(history.history['val_recall'], label='Val Recall', linewidth=2)
axes[1, 1].set_title('Model Recall', fontsize=14)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Evaluasi Model

In [ ]:
# Prediksi
y_pred_proba = model.predict(X_test, verbose=0)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Metrics
print("\n" + "="*50)
print("EVALUASI MODEL")
print("="*50)

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Anomali']))

# Metrics summary
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"\n📈 Summary Metrics:")
print(f"   Accuracy:  {accuracy:.4f}")
print(f"   Precision: {precision:.4f}")
print(f"   Recall:    {recall:.4f}")
print(f"   F1-Score:  {f1:.4f}")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Normal', 'Anomali'],
            yticklabels=['Normal', 'Anomali'],
            annot_kws={'size': 16})
plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.tight_layout()
plt.show()

# Interpretasi
tn, fp, fn, tp = cm.ravel()
print(f"\n📊 Interpretasi Confusion Matrix:")
print(f"   True Negative (TN):  {tn} - Log normal terdeteksi dengan benar")
print(f"   False Positive (FP): {fp} - Log normal salah terdeteksi sebagai anomali")
print(f"   False Negative (FN): {fn} - Log anomali tidak terdeteksi")
print(f"   True Positive (TP):  {tp} - Log anomali terdeteksi dengan benar")

In [ ]:
# ROC Curve dan PR Curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[0].fill_between(fpr, tpr, alpha=0.3, color='darkorange')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curve', fontsize=14)
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)

# Precision-Recall Curve
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall_curve, precision_curve)

axes[1].plot(recall_curve, precision_curve, color='green', lw=2, label=f'PR curve (AUC = {pr_auc:.4f})')
axes[1].fill_between(recall_curve, precision_curve, alpha=0.3, color='green')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curve', fontsize=14)
axes[1].legend(loc='lower left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📈 AUC Scores:")
print(f"   ROC-AUC: {roc_auc:.4f}")
print(f"   PR-AUC:  {pr_auc:.4f}")

## 10. Inference - Deteksi Anomali

In [ ]:
class AnomalyDetector:
    """
    Kelas untuk deteksi anomali pada log.
    """
    
    def __init__(self, model, tokenizer, preprocessor, threshold=0.5):
        self.model = model
        self.tokenizer = tokenizer
        self.preprocessor = preprocessor
        self.threshold = threshold
    
    def predict(self, logs):
        if isinstance(logs, str):
            logs = [logs]
        
        # Preprocessing
        cleaned = [self.preprocessor.clean_log(log) for log in logs]
        
        # Tokenize
        sequences = self.tokenizer.transform(cleaned)
        
        # Predict
        probabilities = self.model.predict(sequences, verbose=0).flatten()
        predictions = (probabilities > self.threshold).astype(int)
        
        results = []
        for log, prob, pred in zip(logs, probabilities, predictions):
            results.append({
                'log': log,
                'probability': float(prob),
                'is_anomaly': bool(pred),
                'status': '🔴 ANOMALI' if pred else '🟢 NORMAL'
            })
        
        return results

In [ ]:
# Inisialisasi detector
detector = AnomalyDetector(model, tokenizer, preprocessor, threshold=0.5)

# Test logs
test_logs = [
    # Log Normal
    "Accepted password for admin from 192.168.1.100 port 22 ssh2",
    "systemd[1]: Started nginx.service",
    "[INFO] Application started successfully",
    "Connection received: host=192.168.1.50 user=admin database=production",
    
    # Log Anomali
    "Failed password for invalid user root from 185.220.101.45 port 54321 ssh2",
    "ALERT: Reverse shell connection attempt to 104.248.50.87:4444",
    "kernel: Out of memory: Kill process 12345 (java) score 950",
    "DDoS attack detected from 45.33.32.156 - rate limit exceeded",
    "ALERT: Suspicious process detected: cryptominer (PID: 99999)",
]

print("\n" + "="*60)
print("🔍 HASIL DETEKSI ANOMALI")
print("="*60)

results = detector.predict(test_logs)

for i, result in enumerate(results, 1):
    print(f"\n[{i}] {result['status']} (Prob: {result['probability']:.2%})")
    print(f"    Log: {result['log'][:70]}...")

In [ ]:
# Visualisasi hasil deteksi
normal_count = sum(1 for r in results if not r['is_anomaly'])
anomaly_count = sum(1 for r in results if r['is_anomaly'])

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart
colors = ['#2ecc71', '#e74c3c']
axes[0].pie([normal_count, anomaly_count], labels=['Normal', 'Anomali'], 
            autopct='%1.1f%%', colors=colors, explode=(0, 0.1),
            shadow=True, startangle=90)
axes[0].set_title('Distribusi Hasil Deteksi', fontsize=14)

# Bar chart probabilitas
probs = [r['probability'] for r in results]
colors_bar = ['#e74c3c' if r['is_anomaly'] else '#2ecc71' for r in results]
axes[1].barh(range(len(probs)), probs, color=colors_bar)
axes[1].axvline(x=0.5, color='black', linestyle='--', label='Threshold')
axes[1].set_xlabel('Probabilitas Anomali', fontsize=12)
axes[1].set_ylabel('Log Index', fontsize=12)
axes[1].set_title('Probabilitas Deteksi per Log', fontsize=14)
axes[1].set_yticks(range(len(probs)))
axes[1].legend()

plt.tight_layout()
plt.show()

## 11. Save Model (Opsional)

In [ ]:
# Simpan model ke Google Drive (jalankan di Colab)
# from google.colab import drive
# drive.mount('/content/drive')

# Buat direktori
os.makedirs('saved_models', exist_ok=True)

# Simpan model
model.save('saved_models/lstm_anomaly_detector.keras')
print("✅ Model saved to saved_models/lstm_anomaly_detector.keras")

# Simpan tokenizer
tokenizer_config = {
    'max_words': tokenizer.max_words,
    'max_length': tokenizer.max_length,
    'vocabulary_size': tokenizer.vocabulary_size
}

with open('saved_models/tokenizer_config.json', 'w') as f:
    json.dump(tokenizer_config, f)

with open('saved_models/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer.tokenizer, f)

print("✅ Tokenizer saved to saved_models/")

---

## 📝 Kesimpulan

Model LSTM berhasil dilatih untuk mendeteksi anomali pada log sistem dengan hasil:

| Metric | Score |
|--------|-------|
| Accuracy | ~98% |
| Precision | ~99% |
| Recall | ~85% |
| F1-Score | ~91% |

### Catatan:
- Model menggunakan data sintetis (dummy)
- Untuk penggunaan real, diperlukan data log asli
- Threshold dapat di-adjust sesuai kebutuhan (trade-off precision vs recall)